In [5]:
using Gurobi
using CSV
using DataFrames
using JuMP

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1192
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190


In [10]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\steph\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [8]:
df = CSV.read("ev_constraints.csv", header = true, missingstring="\n")

,station_1,station_2,station_3,station_4,station_5,station_6
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,2,missing,missing,missing,missing
2,1,2,3,missing,missing,missing
3,1,2,3,missing,missing,missing
4,1,2,3,missing,missing,missing
5,2,3,missing,missing,missing,missing
6,3,missing,missing,missing,missing,missing
7,1,2,3,missing,missing,missing
8,1,2,11,missing,missing,missing
9,2,11,14,missing,missing,missing


In [11]:
num_nodes = 18

m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:num_nodes],Bin)

for row in eachrow(df)
    v = vec(convert(Array, row)) # no transpose necessary
    @constraint(m, sum(x[i] for i in v if !ismissing(i)) >= 1)
end

@objective(m, Min, sum(x[i] for i in 1:num_nodes))


ErrorException: The solver= keyword is no longer available in JuMP 0.19 and later. See the JuMP documentation (http://www.juliaopt.org/JuMP.jl/latest/) for latest syntax.

In [71]:
status = solve(m)

Academic license - for non-commercial use only


:Optimal

In [72]:
getvalue(x)

18-element Array{Float64,1}:
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0

,station_1,station_2,station_3
1,2,missing,missing


In [64]:
 ismissing(df[1,1])

false

In [68]:
m

Feasibility problem with:
 * 114 linear constraints
 * 18 variables: 18 binary
Solver is Gurobi